<a href="https://colab.research.google.com/github/shrav-jally/DL_Lab_Sem6/blob/main/DLLabCMU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
pip install keras

In [ ]:
#upload CMU face dataset to colab
from google.colab import files
uploaded = files.upload()

Saving faces.tar.Z to faces.tar.Z


In [ ]:
#install uncompress tool
!apt-get install ncompress

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  ncompress
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 22.0 kB of archives.
After this operation, 58.4 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncompress amd64 4.2.4.6-5 [22.0 kB]
Fetched 22.0 kB in 0s (48.4 kB/s)
Selecting previously unselected package ncompress.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../ncompress_4.2.4.6-5_amd64.deb ...
Unpacking ncompress (4.2.4.6-5) ...
Setting up ncompress (4.2.4.6-5) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
#uncompress z file
!uncompress faces.tar.Z

In [ ]:
#extract the tar file
!tar -xvf faces.tar

faces/
faces/an2i/
faces/an2i/an2i_straight_neutral_open.pgm
faces/an2i/an2i_left_neutral_open.pgm
faces/an2i/an2i_right_neutral_open.pgm
faces/an2i/an2i_up_neutral_open.pgm
faces/an2i/an2i_straight_happy_open.pgm
faces/an2i/an2i_left_happy_open.pgm
faces/an2i/an2i_right_happy_open.pgm
faces/an2i/an2i_up_happy_open.pgm
faces/an2i/an2i_straight_sad_open.pgm
faces/an2i/an2i_left_sad_open.pgm
faces/an2i/an2i_right_sad_open.pgm
faces/an2i/an2i_up_sad_open.pgm
faces/an2i/an2i_straight_angry_open.pgm
faces/an2i/an2i_left_angry_open.pgm
faces/an2i/an2i_right_angry_open.pgm
faces/an2i/an2i_up_angry_open.pgm
faces/an2i/an2i_straight_neutral_sunglasses.pgm
faces/an2i/an2i_left_neutral_sunglasses.pgm
faces/an2i/an2i_right_neutral_sunglasses.pgm
faces/an2i/an2i_up_neutral_sunglasses.pgm
faces/an2i/an2i_straight_happy_sunglasses.pgm
faces/an2i/an2i_left_happy_sunglasses.pgm
faces/an2i/an2i_right_happy_sunglasses.pgm
faces/an2i/an2i_up_happy_sunglasses.pgm
faces/an2i/an2i_straight_sad_sunglasses.pgm

In [ ]:
#load images properly and extract shape
import numpy as np
import os
from tensorflow.keras.preprocessing import image

dataset_path = "faces"

data = []
labels = []

subjects = sorted(os.listdir(dataset_path))
subject_to_label = {name: idx for idx, name in enumerate(subjects)}

for subject in subjects:
    subject_path = os.path.join(dataset_path, subject)

    # Skip if not a directory
    if not os.path.isdir(subject_path):
        continue

    for img_name in os.listdir(subject_path):
        if img_name.endswith(".pgm"):
            img_path = os.path.join(subject_path, img_name)

            img = image.load_img(img_path, color_mode='grayscale', target_size=(32,30))
            img_array = image.img_to_array(img)

            data.append(img_array)
            labels.append(subject_to_label[subject])

X = np.array(data) / 255.0
y = np.array(labels)

print("Dataset shape:", X.shape)
print("Number of classes:", len(subjects))

Dataset shape: (1872, 32, 30, 1)
Number of classes: 33


In [ ]:
import numpy as np

print("Unique labels:", np.unique(y))
print("Max label:", np.max(y))
print("Min label:", np.min(y))
print("Number of unique labels:", len(np.unique(y)))

Unique labels: [ 1  3  5  7  8 10 11 12 19 22 23 24 25 26 27 28 29 30 31 32]
Max label: 32
Min label: 1
Number of unique labels: 20


In [ ]:
import numpy as np

# Get unique labels
unique_labels = np.unique(y)

# Create mapping: old_label -> new_label (0 to N-1)
label_mapping = {old: new for new, old in enumerate(unique_labels)}

# Apply mapping
y = np.array([label_mapping[label] for label in y])

# Check again
print("New unique labels:", np.unique(y))
print("Number of classes:", len(np.unique(y)))

New unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Number of classes: 20


In [ ]:
#train-test-validation split
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

NUM_CLASSES = len(np.unique(y))  # should be 33

# First split: Train (70%) + Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

# Second split: Validation (15%) + Test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42
)

# One-hot encode
y_train = to_categorical(y_train, NUM_CLASSES)
y_val   = to_categorical(y_val, NUM_CLASSES)
y_test  = to_categorical(y_test, NUM_CLASSES)

print("Train:", X_train.shape)
print("Validation:", X_val.shape)
print("Test:", X_test.shape)

IndexError: index 29 is out of bounds for axis 1 with size 20